1 - Defina o Problema: Escolha um problema que possa ser resolvido com o uso de agentes inteligentes. Descreva o objetivo da aplicação, o público-alvo e as principais funcionalidades planejadas. Detalhe os casos de uso que serão testados para validar a solução.










Problema a ser resolvido: Assistente de pesquisa para artigos científicos e resolução de problemas complexos
Objetivo da aplicação:
Criar um assistente inteligente capaz de ajudar pesquisadores e estudantes a realizar buscas avançadas em diferentes bases de conhecimento, como artigos científicos, Wikipedia, Wolfram Alpha, Google, e DuckDuckGo. Além disso, oferecer suporte para cálculos matemáticos, simulações Python, e respostas a perguntas complexas usando agentes baseados em aprendizado de máquina.

Público-alvo:



Pesquisadores e acadêmicos em busca de informações científicas.
Estudantes de graduação e pós-graduação que precisam resolver problemas complexos e realizar buscas precisas para seus trabalhos.
Profissionais de áreas técnicas e científicas que necessitam de ferramentas de suporte à tomada de decisão.
Principais funcionalidades planejadas:

Busca científica avançada:

Ferramenta para consulta a artigos no ArXiv.
Busca de informações confiáveis em enciclopédias, como a Wikipedia.
Agregador de resultados utilizando DuckDuckGo e Google Search.
Resolução de problemas matemáticos:

Integração com Wolfram Alpha para cálculos simbólicos e numéricos avançados.
Cadeia de cálculos interativos usando a LLMMathChain.
Suporte para programação e simulação:

Um REPL de Python para executar pequenos códigos ou simulações matemáticas diretamente.
Agente de autoaprendizado:

Agente "Self-ask with Search" para responder perguntas complexas combinando múltiplas fontes de informações em tempo real.
Resultados críticos e ferramentas integradas:

Ferramenta de busca crítica ("critical_search") para casos em que perguntas exigem respostas compostas.


2 - Configure o Framework LangChain: Configure o ambiente de desenvolvimento para utilizar LangChain, escolhendo um LLM apropriado (como Gemini ou GPT-4) para sua aplicação.

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-4", verbose=True, streaming=True)


3 - Implemente Ferramentas: Crie e configure no mínimo duas ferramentas externas (tools). Cada ferramenta deve ter nome, descrição e função claramente definidos, e ser integrada ao LangChain.

In [ ]:
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_community.utilities.arxiv import ArxivAPIWrapper
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
from langchain_community.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
from typing import Optional
from langchain_core.language_models import BaseLanguageModel

def load_tools(tool_names: list[str], llm: Optional[BaseLanguageModel] = None):
    """
    Carrega as ferramentas disponíveis para o agente.
    """
    available_tools = {
        "arxiv": ArxivQueryRun(api_wrapper=ArxivAPIWrapper()),
        "wikipedia": WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper()),
        "ddg_search": DuckDuckGoSearchRun(api_wrapper=DuckDuckGoSearchAPIWrapper())
    }
    return [available_tools[name] for name in tool_names if name in available_tools]



4 - Utilize o Framework ReAct: Implemente o raciocínio interativo usando o framework ReAct, garantindo que o agente possa alternar entre raciocinar, tomar ações e observar os resultados para alcançar os objetivos.

In [ ]:
prompt = hub.pull("hwchase17/react")
agent = initialize_agent(tools, llm, agent=AgentType.REACT, verbose=True)


5 - Adicione Memória Conversacional: Configure um sistema de memória conversacional, permitindo que o agente lembre o histórico de interações e utilize esse contexto para gerar respostas mais coerentes.

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import MessagesPlaceholder

def init_memory():
    """Initialize the memory for contextual conversation."""
    return ConversationBufferMemory(
        memory_key="chat_history", return_messages=True, output_key="answer"
    )

MEMORY = init_memory()
CHAT_HISTORY = MessagesPlaceholder(variable_name="chat_history")


6 - Desenvolva a Lógica do Agente: Estruture a lógica de funcionamento do agente, integrando ferramentas, memória e raciocínio interativo para resolver o problema proposto.

In [ ]:
def load_agent(
    tool_names: list[str],
    strategy: ReasoningStrategy = "zero_shot_react"
) -> LLMChain:
    if strategy == "plan_and_solve":
        planner = load_chat_planner(llm)
        executor = load_agent_executor(llm, tools, verbose=True)
        return PlanAndExecute(planner=planner, executor=executor, verbose=True)

    agent = initialize_agent(tools, llm, agent=AgentType.REACT, verbose=True)
    return agent


7 - Projete a Interface do Usuário: Crie uma interface funcional (streamlit ou linha de comando) para que o agente possa ser utilizado. Não é necessário que seja sofisticada, mas deve ser clara e intuitiva.

In [ ]:
st.set_page_config(page_title="Scientific Research Assistant", page_icon=":robot:")

tool_names = st.multiselect(
    "Select tools to assist your research:",
    ["arxiv", "wikipedia", "ddg_search"],
    ["arxiv", "wikipedia", "ddg_search"]
)

if prompt := st.chat_input(placeholder="Enter your research query here..."):
    st.chat_message("user").write(prompt)
    with st.chat_message("assistant"):
        st_callback = StreamlitCallbackHandler(st.container())
        response = agent_chain.invoke({"input": prompt}, {"callbacks": [st_callback]})
        st.write(response["output"])


8 - Teste a Solução: Realize testes práticos utilizando os casos de uso definidos. Inclua uma explicação clara sobre como o agente facilita ou possibilita o trabalho e o que seria necessário fazer sem o uso desse recurso.


9 - Documente o Projeto: Submeta o código no GitHub com um README bem elaborado. O README deve incluir:
Descrição do problema e da solução.
Casos de uso testados e resultados observados.
Instruções claras para execução do código.
Reflexão sobre como o agente facilitou ou possibilitou o trabalho em comparação a uma abordagem manual.

10 - Boas Práticas de Codificação: Certifique-se de que o código segue boas práticas de desenvolvimento, incluindo:
Estrutura modular e organizada.
Comentários explicativos e concisos.
Nomes de variáveis e funções claros e descritivos.
Tratamento adequado de erros.